In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\vooco\\Documents\\Medical_Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vooco\\Documents\\Medical_Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data= load_pdf_file(data='data/')

In [ ]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks


In [9]:
text_chunks= text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\vooco\AppData\Local\Temp\ipykernel_13764\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\vooco\anaconda3\envs\Medichatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY= os.environ.get('GROQ_API_KEY')

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_7GYR8H_NWivjBC6tm7a4midNLUR4D7WUs5L3GnCvfr9612Vjg855AkKBe1BBwFBxxFJ4T")

index_name = "medicalchatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [33]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name= index_name,
    embedding= embeddings,
)

In [34]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding= embeddings,
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [44]:
retrieved_docs = retriever.invoke("Defibition of Antiprotozoal drugs?")

In [45]:
retrieved_docs

[Document(id='b0c2bab3-2070-447f-affa-0ea6af941768', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 311.0, 'page_label': '312', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='antiplatelet drugs\nAntiprotozoal drugs\nDefinition\nAntiprotozoal drugs are medicines that treat infec-\ntions caused by protozoa.\nPurpose\nAntiprotozoal drugs are used to treat a variety of dis-\neases caused by protozoa. Protozoa are animal-like, one-\ncelled animals, such as amoebas. Some are parasites that\ncause infections in the body. African sleeping sickness ,\ngiardiasis, amebiasis, Pneumocystis carinii pneumonia\n(PCP), and malaria are examples of diseases caused by\nprotozoa.\nDescription'),
 Document(id='d8f89a40-ff3a-4509-a843-7d1bf9eabf9a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page

In [39]:
from langchain_groq import ChatGroq
llm= ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.4, max_tokens=500)

In [40]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering task. "
    "Use the following pieces of retrived context to answer "
    "The question. If you don't know the answer, say that you "
    "Don't know. Use three sentences maximum and keep the answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    {
        ("system", system_prompt),
        ("human", "{input}"),
    }
)

In [41]:
question_answer_chain= create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response= rag_chain.invoke({"input": "Defibition of Antiprotozoal drugs?"})
print(response["answer"])

Antiprotozoal drugs are medicines used to treat infections caused by protozoa, which are animal-like, one-celled organisms. Examples of diseases caused by protozoa include African sleeping sickness, giardiasis, amebiasis, Pneumocystis carinii pneumonia (PCP), and malaria. Antiprotozoal drugs come in liquid, tablet, and injectable forms and require a doctor's prescription. The recommended dosage varies depending on the specific drug and medical condition.
